In [1]:
import numpy as np
!pip install nltk

  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 9.6 MB/s eta 0:00:00
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
